# Graft Loss Feature Importance with Monte Carlo Cross-Validation

**Study Replication:** Publication-quality replication with Monte Carlo cross-validation (configurable 100–1000 splits)  
**Updated:** November 21, 2025  
**Hardware:** Optimized for 32-core EC2 instance (1TB RAM)  
**Validation:** Proper evaluation on unseen test data

## Key Changes from Original

✅ **Monte Carlo Cross-Validation** – up to 1000 random 75/25 train/test splits (100-split runs used for faster iteration)  
✅ **Stratified Sampling** - Maintains event distribution  
✅ **Parallel Processing** - Fast execution with furrr/future (≈30 workers)  
✅ **95% Confidence Intervals** - Narrow, precise estimates (tighter with more splits)  
✅ **Realistic C-Indexes** - Expected range 0.70-0.85

## Methodology

This notebook implements the feature selection methodology from the original bcjaeger/graft-loss study:

1. Load data for pediatric heart transplant outcomes
2. Define three time periods:
   - **Original**: 2010-2019 (matches publication)
   - **Full**: 2010-2024 (all available data)
   - **Full No COVID**: 2010-2024 excluding 2020-2023
3. For each period and method:
   - Create 100–1000 stratified train/test splits (75/25)
   - Train model on training set
   - Evaluate on unseen test set
   - Aggregate results across splits
4. Calculate C-index with 95% CI
5. Extract top 20 features

## Expected Runtime (per full 3-period × 3-method run)

- **100 splits (current default):**
  - Local (4 cores): ~2–4 hours (NOT RECOMMENDED)
  - Workstation (16 cores): ~1–2 hours
  - EC2 (32 cores, 1TB RAM): ~1–2 hours ✅ **RECOMMENDED FOR DEVELOPMENT**
- **1000 splits (extended / publication-level):**
  - Local (4 cores): 8–12+ hours (NOT RECOMMENDED)
  - Workstation (16 cores): ~8–16 hours
  - EC2 (32 cores, 1TB RAM): ~10–20 hours ✅ **RECOMMENDED FOR FINAL RESULTS**


## 1. Setup and Configuration

Load required packages and configure parallel processing.


In [ ]:
# Check R version
R.version.string

# Load required packages
library(here)
library(dplyr)
library(readr)
library(survival)
library(ranger)
library(aorsf)
library(catboost)
library(tidyr)
library(purrr)
library(tibble)
library(janitor)
library(haven)
library(riskRegression)
library(prodlim)
library(rsample)    # For MC-CV
library(furrr)      # For parallel processing
library(future)     # For parallel backend
library(progressr)  # For progress bars

cat("✓ All packages loaded successfully\n")


[1] "R version 4.4.3 (2025-02-28)"

here() starts at /home/pgx3874/graft-loss


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test


riskRegression version 2025.09.17

Loading required package: future


Attaching package: ‘future’


The following object is masked from ‘package:survival’:

    cluster




✓ All packages loaded successfully


In [ ]:
# ============================================================
# DEBUG/TEST MODE - Quick testing before full 100-split run
# ============================================================
# Set DEBUG_MODE = TRUE for quick testing (5 splits, ~2-5 min)
# Set DEBUG_MODE = FALSE for full analysis (100 splits, ~1-2 hours on EC2)

DEBUG_MODE <- FALSE  # Change to TRUE for quick test

if (DEBUG_MODE) {
  cat("\n")
  cat("╔════════════════════════════════════════════════════════════════╗\n")
  cat("║                    🔍 DEBUG MODE ENABLED                       ║\n")
  cat("╚════════════════════════════════════════════════════════════════╝\n")
  cat("\n")
  cat("Quick test configuration:\n")
  cat("  • MC-CV Splits: 5 (instead of 100)\n")
  cat("  • Period: Original only (2010-2019)\n")
  cat("  • Trees: Reduced (RSF: 100, AORSF: 50)\n")
  cat("  • Expected time: 2-5 minutes\n")
  cat("  • Purpose: Verify everything works before full run\n")
  cat("\n")
  cat("To run full analysis, set DEBUG_MODE = FALSE\n")
  cat("\n")
}

# Configuration
n_predictors <- 20                                        # Top 20 features
n_trees_rsf <- if (DEBUG_MODE) 100 else 500              # RSF trees (reduced in debug)
n_trees_aorsf <- if (DEBUG_MODE) 50 else 100             # AORSF trees (reduced in debug)
horizon <- 1                                              # 1-year prediction
n_mc_splits <- if (DEBUG_MODE) 5 else 100                # MC-CV splits (5 for debug, 100 for full)
train_prop <- 0.75                                        # 75% training, 25% testing

# Set up parallel processing - EC2 Optimized (32 cores, 1TB RAM)
# Use 30 out of 32 cores (leave 2 for system)
n_workers <- as.integer(Sys.getenv("N_WORKERS", "0"))
if (n_workers < 1) {
  total_cores <- parallel::detectCores()
  n_workers <- max(1, total_cores - 2)
  cat(sprintf("Auto-detected %d cores, using %d workers\n", total_cores, n_workers))
}

cat(sprintf("Setting up parallel processing with %d workers...\n", n_workers))
cat(sprintf("Expected speedup: %dx faster than single core\n", round(n_workers * 0.8)))

# Increase future.globals.maxSize for large MC-CV splits object
# With 1TB RAM on EC2, we can handle large transfers
options(future.globals.maxSize = 20 * 1024^3)  # 20 GB limit (plenty for 100 splits)
cat("Set future.globals.maxSize to 20 GB\n")

plan(multisession, workers = n_workers)

# Create output directory (clean first to ensure fresh start)
output_dir <- here("feature_importance", "outputs")

# Clean existing outputs directory to ensure fresh/clean results
if (dir.exists(output_dir)) {
  # Remove all files in outputs directory
  output_files <- list.files(output_dir, full.names = TRUE, recursive = TRUE, include.dirs = FALSE)
  if (length(output_files) > 0) {
    cat(sprintf("Cleaning %d existing output files...\n", length(output_files)))
    file.remove(output_files)
  }
  # Remove empty subdirectories
  output_dirs <- list.dirs(output_dir, recursive = TRUE, full.names = TRUE)
  output_dirs <- output_dirs[output_dirs != output_dir]  # Don't remove main directory
  for (dir in rev(output_dirs)) {  # Reverse order to remove nested dirs first
    if (length(list.files(dir)) == 0) {
      unlink(dir, recursive = TRUE)
    }
  }
  cat("✓ Output directory cleaned\n")
}

# Create fresh output directory
dir.create(output_dir, showWarnings = FALSE, recursive = TRUE)

cat("Output directory:", output_dir, "\n")
cat(sprintf("MC-CV Configuration: %d splits, %.0f/%.0f train/test split\n", 
            n_mc_splits, train_prop * 100, (1 - train_prop) * 100))
flush.console()


Auto-detected 32 cores, using 30 workers
Setting up parallel processing with 30 workers...
Expected speedup: 24x faster than single core
Set future.globals.maxSize to 20 GB
Output directory: /home/pgx3874/graft-loss/feature_importance/outputs 
MC-CV Configuration: 100 splits, 75/25 train/test split


## 2. Helper Functions

Define functions for data preparation, C-index calculation, and prediction.


In [ ]:
# Prepare modeling data with leakage prevention
prepare_modeling_data <- function(data) {
  # Find time and status columns
  time_col <- intersect(c("time", "outcome_int_graft_loss", "int_graft_loss"), names(data))[1]
  status_col <- intersect(c("status", "outcome_graft_loss", "graft_loss"), names(data))[1]
  
  if (is.na(time_col) || is.na(status_col)) {
    stop("Cannot find time/status columns")
  }
  
  # Rename to standard names
  if (time_col != "time") data <- data %>% rename(time = !!time_col)
  if (status_col != "status") data <- data %>% rename(status = !!status_col)
  
  # Exclude leakage variables and identifier columns
  exclude_exact <- c(
    "ID", "ptid_e", "int_dead", "int_death", "graft_loss", "txgloss", "death", "event",
    "dpricaus", "deathspc", "concod", "age_death", "dlist", "txpl_year",
    "rrace_b", "rrace_a", "rrace_ai", "rrace_pi", "rrace_o", "rrace_un", "race",
    "patsupp", "pmorexam", "papooth", "pacuref", "pishltgr",
    "pathero", "pcadrec", "pcadrem", "pdiffib", "cpathneg",
    "dcardiac", "dneuro", "dreject", "dsecaccs", "dpriaccs",
    "dconmbld", "dconmal", "dconcard", "dconneur", "dconrej",
    "dmajbld", "dmalcanc"
  )
  
  exclude_prefixes <- c("dtx_", "cc_", "dcon", "dpri", "dsec", "dmaj", "sd")
  
  exclude_by_prefix <- character(0)
  for (prefix in exclude_prefixes) {
    exclude_by_prefix <- c(exclude_by_prefix, 
                           names(data)[startsWith(names(data), prefix)])
  }
  
  exclude_all <- unique(c(exclude_exact, exclude_by_prefix))
  data <- data %>% select(-any_of(exclude_all))
  
  # Median imputation for numeric variables
  numeric_vars <- names(data)[sapply(data, is.numeric) & names(data) != "time" & names(data) != "status"]
  for (var in numeric_vars) {
    if (any(is.na(data[[var]]))) {
      median_val <- median(data[[var]], na.rm = TRUE)
      data[[var]][is.na(data[[var]])] <- median_val
    }
  }
  
  # Mode imputation for categorical variables
  categorical_vars <- names(data)[sapply(data, function(x) is.factor(x) | is.character(x))]
  for (var in categorical_vars) {
    if (any(is.na(data[[var]]))) {
      mode_val <- names(sort(table(data[[var]]), decreasing = TRUE))[1]
      data[[var]][is.na(data[[var]])] <- mode_val
    }
  }
  
  # Remove constant columns
  constant_cols <- names(data)[sapply(data, function(x) length(unique(na.omit(x))) <= 1)]
  if (length(constant_cols) > 0) {
    data <- data %>% select(-any_of(constant_cols))
  }
  
  # Convert character to factor
  data <- data %>% mutate(across(where(is.character), as.factor))
  
  return(data)
}

cat("✓ prepare_modeling_data() defined\n")


✓ prepare_modeling_data() defined


In [ ]:
# C-index calculation
calculate_cindex <- function(time, status, risk_scores, horizon = NULL) {
  valid_idx <- !is.na(time) & !is.na(status) & !is.na(risk_scores) &
               is.finite(time) & is.finite(risk_scores) & time > 0
  
  time   <- as.numeric(time[valid_idx])
  status <- as.numeric(status[valid_idx])
  risk   <- as.numeric(risk_scores[valid_idx])
  
  n <- length(time)
  events <- sum(status == 1)
  
  if (n < 10 || events < 1 || length(unique(risk)) == 1) {
    return(list(cindex_td = NA_real_, cindex_ti = NA_real_))
  }
  
  # Time-independent C-index (Harrell's)
  num_conc_ti <- 0
  num_disc_ti <- 0
  num_ties_ti <- 0
  
  for (i in seq_len(n)) {
    if (status[i] != 1) next
    for (j in seq_len(n)) {
      if (i == j) next
      if (time[i] < time[j]) {
        if (risk[i] > risk[j]) {
          num_conc_ti <- num_conc_ti + 1
        } else if (risk[i] < risk[j]) {
          num_disc_ti <- num_disc_ti + 1
        } else {
          num_ties_ti <- num_ties_ti + 1
        }
      }
    }
  }
  
  denom_ti <- num_conc_ti + num_disc_ti + num_ties_ti
  cindex_ti <- if (denom_ti > 0) (num_conc_ti + 0.5 * num_ties_ti) / denom_ti else NA_real_
  
  # Time-dependent C-index
  cindex_td <- tryCatch({
    score_data <- data.frame(time = time, status = status)
    pred_matrix <- matrix(risk, ncol = 1)
    
    evaluation <- riskRegression::Score(
      object = list(Model = pred_matrix),
      formula = Surv(time, status) ~ 1,
      data = score_data,
      times = if (!is.null(horizon)) horizon else median(time[status == 1]),
      summary = "risks",
      metrics = "auc",
      se.fit = FALSE
    )
    
    as.numeric(evaluation$AUC$score$AUC[1])
  }, error = function(e) cindex_ti)
  
  return(list(cindex_td = cindex_td, cindex_ti = cindex_ti))
}

# ranger_predictrisk function
ranger_predictrisk <- function(object, newdata, times) {
  preds <- predict(object, data = newdata, type = "response")
  if (is.null(preds$survival)) {
    stop("ranger prediction did not return survival probabilities")
  }
  
  surv_matrix <- preds$survival
  time_points <- preds$unique.death.times
  closest_idx <- which.min(abs(time_points - times))
  risk_scores <- 1 - surv_matrix[, closest_idx]
  
  return(as.numeric(risk_scores))
}

cat("✓ calculate_cindex() and ranger_predictrisk() defined\n")


✓ calculate_cindex() and ranger_predictrisk() defined


## 3. Monte Carlo Cross-Validation Function

Main function that runs MC-CV for a single method and period.


In [ ]:
# Run MC-CV for a single method and time period
run_mc_cv_method <- function(data, method, period_name, mc_splits) {
  
  cat(sprintf("\n=== Running MC-CV for %s (%s) ===\n", method, period_name))
  cat(sprintf("Splits: %d | Train: %.0f%% | Test: %.0f%%\n", 
              n_mc_splits, train_prop * 100, (1 - train_prop) * 100))
  flush.console()
  
  split_ids <- seq_len(n_mc_splits)
  
  # Run splits in parallel with progress bar
  with_progress({
    p <- progressor(steps = n_mc_splits)
    
    results <- future_map(split_ids, function(split_id) {
      p()  # Update progress
      
      # Get train/test data from split
      split <- mc_splits$splits[[split_id]]
      train_data <- rsample::analysis(split)
      test_data <- rsample::assessment(split)
      
      # Train and evaluate
      model <- NULL
      predictions <- NULL
      feature_importance <- NULL
      
      tryCatch({
        if (method == "RSF") {
          model <- ranger(
            Surv(time, status) ~ .,
            data = train_data,
            num.trees = n_trees_rsf,
            importance = "permutation",
            min.node.size = 20,
            splitrule = "extratrees",
            num.random.splits = 10
          )
          
          predictions <- ranger_predictrisk(model, test_data, horizon)
          feature_importance <- model$variable.importance
          
        } else if (method == "AORSF") {
          # Remove constant columns from training data (can occur after train/test split)
          constant_cols <- names(train_data)[sapply(train_data, function(x) {
            if (is.numeric(x)) {
              length(unique(na.omit(x))) <= 1
            } else {
              length(unique(na.omit(x))) <= 1
            }
          })]
          constant_cols <- setdiff(constant_cols, c("time", "status"))
          if (length(constant_cols) > 0) {
            train_data <- train_data %>% select(-any_of(constant_cols))
            test_data <- test_data %>% select(-any_of(constant_cols))
          }
          
          model <- aorsf::orsf(
            data = train_data,
            formula = Surv(time, status) ~ .,
            n_tree = n_trees_aorsf,
            na_action = 'impute_meanmode'
          )
          
          pred_obj <- predict(model, new_data = test_data, 
                              pred_type = 'risk', pred_horizon = horizon)
          predictions <- if (is.matrix(pred_obj)) as.numeric(pred_obj[, 1]) else as.numeric(pred_obj)
          feature_importance <- aorsf::orsf_vi_permute(model)
          
        } else if (method == "CatBoost") {
          # Remove constant columns from training data (can occur after train/test split)
          constant_cols <- names(train_data)[sapply(train_data, function(x) {
            if (is.numeric(x)) {
              length(unique(na.omit(x))) <= 1
            } else {
              length(unique(na.omit(x))) <= 1
            }
          })]
          constant_cols <- setdiff(constant_cols, c("time", "status"))
          if (length(constant_cols) > 0) {
            train_data <- train_data %>% select(-any_of(constant_cols))
            test_data <- test_data %>% select(-any_of(constant_cols))
          }
          
          train_pool <- catboost.load_pool(
            data = train_data %>% select(-time, -status),
            label = train_data$time
          )
          
          test_pool <- catboost.load_pool(
            data = test_data %>% select(-time, -status)
          )
          
          # CatBoost configuration: single-threaded inside each R worker to avoid
          # logger/thread-safety issues, and quiet logging in parallel runs.
          params <- list(
            loss_function  = 'Cox',
            iterations     = 100,
            learning_rate  = 0.1,
            depth          = 6,
            thread_count   = 1,
            logging_level  = 'Silent',
            verbose        = 0L
          )
          
          model <- catboost.train(train_pool, params = params)
          predictions <- catboost.predict(model, test_pool)
          
          # Get feature importance - CatBoost returns a matrix with rownames as feature names
          # IMPORTANT: catboost.get_feature_importance() returns a matrix (not a named vector)
          # - Values are in the first column: importance_matrix[, 1]
          # - Feature names are in rownames: rownames(importance_matrix)
          # Convert to named vector for consistency with RSF and AORSF (which return named vectors directly)
          importance_matrix <- catboost.get_feature_importance(model)
          feature_importance <- as.numeric(importance_matrix[, 1])
          names(feature_importance) <- rownames(importance_matrix)
        }
        
        # Calculate C-index on TEST data
        cindex_result <- calculate_cindex(
          time = test_data$time,
          status = test_data$status,
          risk_scores = predictions,
          horizon = horizon
        )
        
        return(list(
          split_id = split_id,
          cindex_td = cindex_result$cindex_td,
          cindex_ti = cindex_result$cindex_ti,
          feature_importance = feature_importance,
          n_train = nrow(train_data),
          n_test = nrow(test_data),
          success = TRUE
        ))
        
      }, error = function(e) {
        return(list(
          split_id = split_id,
          cindex_td = NA_real_,
          cindex_ti = NA_real_,
          feature_importance = NULL,
          n_train = nrow(train_data),
          n_test = nrow(test_data),
          success = FALSE,
          error = e$message
        ))
      })
    }, .options = furrr_options(
      seed = TRUE,
      packages = c(
        "dplyr", "purrr", "tibble", "rsample",
        "ranger", "aorsf", "catboost",
        "riskRegression", "prodlim"
      )
    ))
  })
  
  # Aggregate results
  successful_splits <- Filter(function(x) x$success, results)
  n_successful <- length(successful_splits)
  
  cat(sprintf("Successful splits: %d / %d\n", n_successful, n_mc_splits))
  flush.console()
  
  # If all splits failed (e.g., CatBoost configuration/data issue),
  # do not stop the whole analysis. Instead, log a warning and return
  # an empty/NA summary so that other methods and periods can continue.
  if (n_successful == 0) {
    error_splits <- Filter(function(x) !is.null(x$error), results)
    if (length(error_splits) > 0) {
      first_error <- error_splits[[1]]$error
      cat(sprintf("WARNING: example error for %s (%s): %s\n",
                  method, period_name, first_error))
    }
    cat(sprintf("WARNING: All MC-CV splits failed for %s (%s). Skipping this method.\n",
                method, period_name))
    flush.console()
    
    return(list(
      method = method,
      period = period_name,
      n_splits = n_mc_splits,
      n_successful = 0,
      cindex_td_mean = NA_real_,
      cindex_td_sd = NA_real_,
      cindex_td_ci_lower = NA_real_,
      cindex_td_ci_upper = NA_real_,
      cindex_ti_mean = NA_real_,
      cindex_ti_sd = NA_real_,
      cindex_ti_ci_lower = NA_real_,
      cindex_ti_ci_upper = NA_real_,
      top_features = numeric(0)
    ))
  }
  
  # Extract C-indexes
  cindex_td_values <- sapply(successful_splits, function(x) x$cindex_td)
  cindex_ti_values <- sapply(successful_splits, function(x) x$cindex_ti)
  
  cindex_td_values <- cindex_td_values[!is.na(cindex_td_values)]
  cindex_ti_values <- cindex_ti_values[!is.na(cindex_ti_values)]
  
  # Aggregate feature importance
  # All methods (RSF, AORSF, CatBoost) return named numeric vectors
  all_feature_names <- unique(unlist(lapply(successful_splits, function(x) {
    if (is.null(x$feature_importance)) return(NULL)
    names(x$feature_importance)
  })))
  
  aggregated_importance <- sapply(all_feature_names, function(feature) {
    importances <- sapply(successful_splits, function(x) {
      if (is.null(x$feature_importance)) return(NA_real_)
      if (feature %in% names(x$feature_importance)) {
        return(as.numeric(x$feature_importance[feature]))
      }
      return(NA_real_)
    })
    mean(importances, na.rm = TRUE)
  })
  
  # Ensure aggregated_importance is a numeric vector
  aggregated_importance <- as.numeric(aggregated_importance)
  names(aggregated_importance) <- all_feature_names
  
  top_features <- sort(aggregated_importance, decreasing = TRUE)[1:min(n_predictors, length(aggregated_importance))]
  
  # Calculate statistics
  results_summary <- list(
    method = method,
    period = period_name,
    n_splits = n_mc_splits,
    n_successful = n_successful,
    cindex_td_mean = mean(cindex_td_values, na.rm = TRUE),
    cindex_td_sd = sd(cindex_td_values, na.rm = TRUE),
    cindex_td_ci_lower = quantile(cindex_td_values, 0.025, na.rm = TRUE),
    cindex_td_ci_upper = quantile(cindex_td_values, 0.975, na.rm = TRUE),
    cindex_ti_mean = mean(cindex_ti_values, na.rm = TRUE),
    cindex_ti_sd = sd(cindex_ti_values, na.rm = TRUE),
    cindex_ti_ci_lower = quantile(cindex_ti_values, 0.025, na.rm = TRUE),
    cindex_ti_ci_upper = quantile(cindex_ti_values, 0.975, na.rm = TRUE),
    top_features = top_features
  )
  
  # Print summary
  cat(sprintf("\n--- Results for %s (%s) ---\n", method, period_name))
  cat(sprintf("Time-Dependent C-Index: %.4f ± %.4f (95%% CI: %.4f - %.4f)\n",
              results_summary$cindex_td_mean,
              results_summary$cindex_td_sd,
              results_summary$cindex_td_ci_lower,
              results_summary$cindex_td_ci_upper))
  cat(sprintf("Time-Independent C-Index: %.4f ± %.4f (95%% CI: %.4f - %.4f)\n",
              results_summary$cindex_ti_mean,
              results_summary$cindex_ti_sd,
              results_summary$cindex_ti_ci_lower,
              results_summary$cindex_ti_ci_upper))
  # Display top 10 features sorted alphabetically for easier comparison
  top10_features <- names(top_features)[1:min(10, length(top_features))]
  top10_features_sorted <- sort(top10_features)
  cat(sprintf("Top 10 features (alphabetical): %s\n", paste(top10_features_sorted, collapse = ", ")))
  flush.console()
  
  return(results_summary)
}

cat("✓ run_mc_cv_method() defined\n")


✓ run_mc_cv_method() defined


## 4. Load Data

In [ ]:
# Find and load SAS file
sas_path_local <- here("data", "phts_txpl_ml.sas7bdat")
sas_path_external <- here("graft-loss-parallel-processing", "data", "phts_txpl_ml.sas7bdat")
sas_path_graft_loss <- here("graft-loss", "data", "phts_txpl_ml.sas7bdat")

sas_path <- NULL
for (path in c(sas_path_local, sas_path_external, sas_path_graft_loss)) {
  if (file.exists(path)) {
    sas_path <- path
    break
  }
}

if (is.null(sas_path)) {
  stop("Cannot find phts_txpl_ml.sas7bdat in any location")
}

cat("Loading data from:", sas_path, "\n")

# Load data
phts_base <- haven::read_sas(sas_path) %>%
  filter(TXPL_YEAR >= 2010) %>%
  janitor::clean_names() %>%
  rename(
    outcome_int_graft_loss = int_graft_loss,
    outcome_graft_loss = graft_loss
  ) %>%
  mutate(
    ID = 1:n(),
    across(.cols = where(is.character), ~ ifelse(.x %in% c("", "unknown", "missing"), NA_character_, .x)),
    across(.cols = where(is.character), as.factor),
    tx_mcsd = if ('txnomcsd' %in% names(.)) {
      if_else(txnomcsd == 'yes', 0, 1)
    } else if ('txmcsd' %in% names(.)) {
      txmcsd
    } else {
      NA_real_
    }
  )

cat(sprintf("✓ Loaded data: %d rows, %d columns\n", nrow(phts_base), ncol(phts_base)))


Loading data from: /home/pgx3874/graft-loss/data/phts_txpl_ml.sas7bdat 
✓ Loaded data: 5835 rows, 478 columns


In [ ]:
# Define time periods
periods <- list()
periods$original <- phts_base %>% filter(txpl_year >= 2010 & txpl_year <= 2019)
periods$full <- phts_base %>% filter(txpl_year >= 2010)
periods$full_no_covid <- phts_base %>% filter(txpl_year >= 2010 & !(txpl_year >= 2020 & txpl_year <= 2023))

# Print summary
for (period_name in names(periods)) {
  period_data <- periods[[period_name]]
  n_events <- sum(period_data$outcome_graft_loss, na.rm = TRUE)
  event_rate <- 100 * n_events / nrow(period_data)
  
  cat(sprintf("Period: %s | N: %d | Events: %d (%.2f%%)\n", 
              period_name, nrow(period_data), n_events, event_rate))
}

cat("\n✓ Time periods defined\n")


Period: original | N: 4036 | Events: 768 (19.03%)
Period: full | N: 5835 | Events: 939 (16.09%)
Period: full_no_covid | N: 4196 | Events: 774 (18.45%)

✓ Time periods defined


## 5. Run Analysis

Execute MC-CV for all methods and periods.

- **100 splits (default full run):** Typically ~1–2 hours on a 32-core EC2 instance, longer on smaller machines.
- **1000 splits (extended run):** Linearly more expensive; expect roughly 8–10× the 100-split time.

**Note:** You can run each period separately by uncommenting only one `period_name` at a time, or reduce the number of methods/periods to shorten runtime further.


In [ ]:
# Select periods and methods to run
# In DEBUG_MODE, only run original period for speed
period_names <- if (DEBUG_MODE) {
  c("original")  # Debug: just one period (~2-5 min)
} else {
  c("original", "full", "full_no_covid")  # Full: all periods (~30-45 min)
}

# Methods to run. Include CatBoost (with single-threaded, quiet config)
# alongside RSF and AORSF.
method_names <- c("RSF", "CatBoost", "AORSF")

# Store all results
all_results <- list()

# Run analysis
for (period_name in period_names) {
  cat(sprintf("\n========================================\n"))
  cat(sprintf("Processing Period: %s\n", period_name))
  cat(sprintf("========================================\n"))
  flush.console()
  
  # Prepare data
  period_data <- prepare_modeling_data(periods[[period_name]])
  
  cat(sprintf("Period data: %d rows, %d columns\n", nrow(period_data), ncol(period_data)))
  cat(sprintf("Events: %d (%.2f%%)\n", sum(period_data$status), 
              100 * sum(period_data$status) / nrow(period_data)))
  
  # Create MC-CV splits (stratified by outcome)
  cat(sprintf("Creating %d MC-CV splits (stratified)...\n", n_mc_splits))
  flush.console()
  mc_splits <- mc_cv(
    data = period_data,
    prop = train_prop,
    times = n_mc_splits,
    strata = status
  )
  
  # Run each method
  period_results <- list()
  
  for (method in method_names) {
    result <- run_mc_cv_method(period_data, method, period_name, mc_splits)
    period_results[[method]] <- result
    
    # Save top features (sorted alphabetically for easier comparison)
    top_features_df <- tibble(
      feature = names(result$top_features),
      importance = as.numeric(result$top_features),
      cindex_td = result$cindex_td_mean,
      cindex_ti = result$cindex_ti_mean
    ) %>%
      arrange(feature)  # Sort alphabetically for easier visual comparison
    
    output_file <- file.path(output_dir, sprintf("%s_%s_top20.csv", 
                                                  period_name, tolower(method)))
    write_csv(top_features_df, output_file)
    cat(sprintf("✓ Saved: %s\n", basename(output_file)))
  }
  
  all_results[[period_name]] <- period_results
}

cat("\n✓ Analysis complete for all periods!\n")



Processing Period: original
Period data: 4036 rows, 390 columns
Events: 768 (19.03%)
Creating 100 MC-CV splits (stratified)...

=== Running MC-CV for RSF (original) ===
Splits: 100 | Train: 75% | Test: 25%
Computing permutation importance.. Progress: 10%. Estimated remaining time: 4 minutes, 35 seconds.
Computing permutation importance.. Progress: 17%. Estimated remaining time: 4 minutes, 59 seconds.
Computing permutation importance.. Progress: 23%. Estimated remaining time: 5 minutes, 7 seconds.
Computing permutation importance.. Progress: 28%. Estimated remaining time: 5 minutes, 26 seconds.
Computing permutation importance.. Progress: 33%. Estimated remaining time: 5 minutes, 22 seconds.
Computing permutation importance.. Progress: 38%. Estimated remaining time: 5 minutes, 16 seconds.
Computing permutation importance.. Progress: 42%. Estimated remaining time: 5 minutes, 2 seconds.
Computing permutation importance.. Progress: 47%. Estimated remaining time: 4 minutes, 47 seconds.
Com

## 6. Save Summary Results

Create summary tables with C-index comparisons and statistics.


In [ ]:
# Create C-index comparison table
cindex_comparison <- map_df(period_names, function(period) {
  map_df(method_names, function(method) {
    result <- all_results[[period]][[method]]
    tibble(
      period = period,
      method = method,
      cindex_td_mean = result$cindex_td_mean,
      cindex_td_sd = result$cindex_td_sd,
      cindex_td_ci_lower = result$cindex_td_ci_lower,
      cindex_td_ci_upper = result$cindex_td_ci_upper,
      cindex_ti_mean = result$cindex_ti_mean,
      cindex_ti_sd = result$cindex_ti_sd,
      cindex_ti_ci_lower = result$cindex_ti_ci_lower,
      cindex_ti_ci_upper = result$cindex_ti_ci_upper,
      n_splits = result$n_successful
    )
  })
})

write_csv(cindex_comparison, file.path(output_dir, "cindex_comparison_mc_cv.csv"))
cat("✓ Saved: cindex_comparison_mc_cv.csv\n")

# Display the table
print(cindex_comparison)


✓ Saved: cindex_comparison_mc_cv.csv
# A tibble: 9 × 11
  period        method   cindex_td_mean cindex_td_sd cindex_td_ci_lower
  <chr>         <chr>             <dbl>        <dbl>              <dbl>
1 original      RSF               0.778       0.0243              0.718
2 original      CatBoost          0.476       0.0327              0.408
3 original      AORSF             0.749       0.0247              0.696
4 full          RSF               0.787       0.0195              0.747
5 full          CatBoost          0.502       0.0271              0.448
6 full          AORSF             0.760       0.0207              0.713
7 full_no_covid RSF               0.777       0.0194              0.735
8 full_no_covid CatBoost          0.482       0.0349              0.409
9 full_no_covid AORSF             0.747       0.0219              0.706
# ℹ 6 more variables: cindex_td_ci_upper <dbl>, cindex_ti_mean <dbl>,
#   cindex_ti_sd <dbl>, cindex_ti_ci_lower <dbl>, cindex_ti_ci_upper <dbl>,
#   n_

In [ ]:
# Create summary statistics
summary_stats <- map_df(period_names, function(period) {
  period_data <- periods[[period]]
  tibble(
    period = period,
    n_patients = nrow(period_data),
    n_events = sum(period_data$outcome_graft_loss, na.rm = TRUE),
    event_rate = 100 * sum(period_data$outcome_graft_loss, na.rm = TRUE) / nrow(period_data)
  )
})

write_csv(summary_stats, file.path(output_dir, "summary_statistics_mc_cv.csv"))
cat("✓ Saved: summary_statistics_mc_cv.csv\n")

# Display the table
print(summary_stats)


✓ Saved: summary_statistics_mc_cv.csv
# A tibble: 3 × 4
  period        n_patients n_events event_rate
  <chr>              <int>    <dbl>      <dbl>
1 original            4036      768       19.0
2 full                5835      939       16.1
3 full_no_covid       4196      774       18.4


In [ ]:
# Close parallel processing
plan(sequential)

cat("\n========================================\n")
cat("Analysis Complete!\n")
cat("========================================\n")
cat(sprintf("Output directory: %s\n", output_dir))
cat(sprintf("MC-CV splits: %d\n", n_mc_splits))
cat(sprintf("Train/Test ratio: %.0f/%.0f\n", train_prop * 100, (1 - train_prop) * 100))
cat("\nResults show C-indexes with 95% confidence intervals\n")
cat("based on", n_mc_splits, "independent train/test splits.\n\n")
cat("✓ All files saved successfully!\n")



Analysis Complete!
Output directory: /home/pgx3874/graft-loss/feature_importance/outputs
MC-CV splits: 100
Train/Test ratio: 75/25

Results show C-indexes with 95% confidence intervals
based on 100 independent train/test splits.

✓ All files saved successfully!


## 7. Visualize Results

In [ ]:
# Create visualization plots for feature importance and C-index analysis
# Uses the updated create_visualizations.R script with improved normalization procedure
# Creates: Feature importance heatmap, C-index heatmap, scaled bar chart, and C-index table

library(here)

# Source the visualization script
source(here("feature_importance", "create_visualizations.R"))

# Run visualizations with the output directory
run_visualizations(output_dir = here("feature_importance", "outputs"))


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.2     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.1.0     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
here() starts at /home/pgx3874/graft-loss

Rows: 9 Columns: 11
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): period, method
dbl (9): cindex_td_mean, cindex_td_sd, cindex_td_ci_lower, cindex_td_ci_uppe...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.



Loading feature importance files...


Rows: 20 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): feature
dbl (3): importance, cindex_td, cindex_ti

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


✓ Loaded: original_rsf_top20.csv


Rows: 20 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): feature
dbl (3): importance, cindex_td, cindex_ti

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


...

✓ Loaded: full_no_covid_rsf_top20.csv


Rows: 20 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): feature
dbl (3): importance, cindex_td, cindex_ti

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


✓ Loaded: full_no_covid_catboost_top20.csv


Rows: 20 Columns: 4
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (1): feature
dbl (3): importance, cindex_td, cindex_ti

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


✓ Loaded: full_no_covid_aorsf_top20.csv

Loaded 9/9 expected feature files
Total features loaded: 180
Unique features: 46

Debug: Methods in all_features: RSF CatBoost AORSF 
Debug: Periods in all_features: original full full_no_covid 
Debug: Sample of all_features:
# A tibble: 10 × 4
   feature      period   method importance
   <chr>        <chr>    <chr>       <dbl>
 1 chd_hlh      original RSF      0.000932
 2 cpbypass     original RSF      0.00475 
 3 dlist        original RSF      0.00812 
 4 donor_age    original RSF      0.000591
 5 height_donor original RSF      0.000564
 6 height_txpl  original RSF      0.000510
 7 hxsurg       original RSF      0.00208 
 8 lbun_r       original RSF      0.000606
 9 lsbaosat     original RSF      0.00132 
10 lsfprab      original RSF      0.00106 

Debug: Feature matrix summary (before normalization):
Methods in feature_matrix: 1 2 3 
Non-zero importance values by method:
# A tibble: 3 × 4
  method   n_nonzero max_importance mean_importance
 

## 8. Sync Results and Code to S3

Sync output files and code (notebook + R script) to S3 bucket. 
- Outputs: CSV results files
- Code: Notebook and R script for reproducibility

In [ ]:
# Sync outputs and code to S3
# On EC2, we're in the feature_importance directory
s3_bucket <- "s3://uva-private-data-lake/graft-loss/feature_importance/"

# Find AWS CLI (check common locations - EC2 typically has it in /usr/local/bin or /usr/bin)
aws_cmd <- Sys.which("aws")
if (aws_cmd == "") {
  # Try common EC2 installation paths
  aws_paths <- c(
    "/usr/local/bin/aws",
    "/usr/bin/aws",
    "/home/ec2-user/.local/bin/aws"
  )
  aws_cmd <- NULL
  for (path in aws_paths) {
    if (file.exists(path)) {
      aws_cmd <- path
      break
    }
  }
  if (is.null(aws_cmd)) {
    stop("AWS CLI not found. Please install AWS CLI or ensure it's in your PATH.")
  }
}

cat("Syncing outputs and code to S3...\n")
cat("Source: feature_importance/ directory\n")
cat("Destination:", s3_bucket, "\n")
cat("AWS CLI:", aws_cmd, "\n\n")

# Get current directory (should be feature_importance)
current_dir <- getwd()
if (!grepl("feature_importance", current_dir)) {
  warning("Current directory doesn't appear to be feature_importance. Double-check sync destination.")
}

# Sync feature_importance directory (includes outputs/ and code files)
# Explicitly include notebook, R scripts, README files, and outputs directory
# Exclude temporary files, checkpoints, and unnecessary directories
# Note: --delete flag removed for safety (won't delete files in S3 that don't exist locally)
# Include patterns are processed before exclude patterns, then exclude everything else
sync_cmd <- sprintf(
  '"%s" s3 sync "%s" %s --include "*.ipynb" --include "*.R" --include "README*.md" --include "outputs/**" --exclude "*checkpoint*" --exclude "*.tmp" --exclude "*.ipynb_checkpoints/*" --exclude "*.RData" --exclude "*.Rhistory" --exclude ".Rproj.user/*" --exclude "catboost_info/*" --exclude "*.log" --exclude "*"',
  aws_cmd,
  current_dir,
  s3_bucket
)

cat("Running:", sync_cmd, "\n\n")
result <- system(sync_cmd)

if (result == 0) {
  cat("✓ Successfully synced outputs and code to S3\n")
  cat("  - Outputs:", file.path(output_dir), "\n")
  cat("  - Code: *.ipynb, *.R, README*.md\n")
} else {
  warning(sprintf("S3 sync returned exit code %d. Check AWS credentials and permissions.", result))
}


Syncing outputs and code to S3...
Source: feature_importance/ directory
Destination: s3://uva-private-data-lake/graft-loss/feature_importance/ 
AWS CLI: /usr/local/bin/aws 

Running: "/usr/local/bin/aws" s3 sync "/home/pgx3874/graft-loss/feature_importance" s3://uva-private-data-lake/graft-loss/feature_importance/ --exclude "*checkpoint*" --exclude "*.tmp" --exclude "*.ipynb_checkpoints/*" --exclude "*.RData" --exclude "*.Rhistory" --exclude ".Rproj.user/*" --exclude "catboost_info/*" --exclude "*.log" 

✓ Successfully synced outputs and code to S3
  - Outputs: /home/pgx3874/graft-loss/feature_importance/outputs 
  - Code: *.ipynb, *.R, README*.md


## 9. Shutdown EC2

In [ ]:

# Shutdown EC2 instance after analysis completes
# Set SHUTDOWN_EC2 = TRUE to enable, FALSE to disable
SHUTDOWN_EC2 <- TRUE  # Change to TRUE to enable auto-shutdown

if (SHUTDOWN_EC2) {
  cat("\n========================================\n")
  cat("Shutting down EC2 instance...\n")
  cat("========================================\n")
  
  # Get instance ID from EC2 metadata service
  instance_id <- tryCatch({
    system("curl -s http://169.254.169.254/latest/meta-data/instance-id", intern = TRUE)
  }, error = function(e) {
    cat("Warning: Could not retrieve instance ID from metadata service.\n")
    cat("If running on EC2, check that metadata service is accessible.\n")
    return(NULL)
  })
  
  if (!is.null(instance_id) && length(instance_id) > 0 && nchar(instance_id[1]) > 0) {
    instance_id <- instance_id[1]
    cat(sprintf("Instance ID: %s\n", instance_id))
    
    # Find AWS CLI
    aws_cmd <- Sys.which("aws")
    if (aws_cmd == "") {
      aws_paths <- c(
        "/usr/local/bin/aws",
        "/usr/bin/aws",
        "/home/ec2-user/.local/bin/aws"
      )
      aws_cmd <- NULL
      for (path in aws_paths) {
        if (file.exists(path)) {
          aws_cmd <- path
          break
        }
      }
    }
    
    if (!is.null(aws_cmd) && aws_cmd != "") {
      # Stop the instance (use terminate-instances for permanent deletion)
      shutdown_cmd <- sprintf(
        '"%s" ec2 stop-instances --instance-ids %s',
        aws_cmd,
        instance_id
      )
      
      cat("Running:", shutdown_cmd, "\n")
      result <- system(shutdown_cmd)
      
      if (result == 0) {
        cat("✓ EC2 instance stop command sent successfully\n")
        cat("Instance will stop in a few moments.\n")
        cat("Note: This is a STOP (not terminate), so you can restart it later.\n")
      } else {
        warning(sprintf("EC2 stop command returned exit code %d. Check AWS credentials and permissions.", result))
      }
    } else {
      cat("Warning: AWS CLI not found. Cannot shutdown instance.\n")
      cat("Install AWS CLI or ensure it's in your PATH.\n")
    }
  } else {
    cat("Warning: Could not determine instance ID. Skipping shutdown.\n")
    cat("If you want to shutdown manually, use:\n")
    cat("  aws ec2 stop-instances --instance-ids <your-instance-id>\n")
  }
} else {
  cat("\n========================================\n")
  cat("EC2 Auto-Shutdown: DISABLED\n")
  cat("========================================\n")
  cat("To enable auto-shutdown, set SHUTDOWN_EC2 = TRUE in this cell.\n")
  cat("Instance will continue running.\n")
}
